In [126]:
import requests
import pandas as pd
import joblib
import json


In [127]:
header_list = ['datetime', 'road_number', 'km', 'direction', 'all_units', 'inflow_units',
               'outflow_unit', 'samecell_units', 'avg_speed', 'max_speed', 'avg_traveltime', 'max_traveltime']

In [128]:
num_cols = ['all_units', 'inflow_units',
            'portion_speed', 'max_speed',
            'avg_traveltime', 'max_traveltime']

In [129]:
df_km127 = pd.read_csv("../dataset/latlon_km127.csv")

In [130]:
def map_traffic_with_latlon(df):
    df['lat'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values) > 0 else 0, axis=1)
    df['lon'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values) > 0 else 0, axis=1)
    return df

In [131]:
road = 2
direction = "in"
sus_cluster = 1

In [132]:
gnb_model = joblib.load(f'../Models/GaussianNBModelRoad{road}{direction}.joblib')

In [133]:
road_data = pd.read_csv("http://analytics2.dlt.transcodeglobal.com/cell_data/current_celldata.csv",names=header_list)

In [134]:
dfm = pd.read_csv(f'../dataset/road{road}-{direction}-mffs.csv')

In [135]:
road_data = road_data[(road_data['road_number'] == road)]

In [136]:
road_data['mffs'] = road_data['km'].map(dfm.set_index('km')['mffs'])

In [137]:
road_data = map_traffic_with_latlon(road_data)

In [138]:
road2in_data = road_data[(road_data['road_number']==road) & (road_data['direction']=='in')]

In [139]:
road2in_data['portion_speed'] = road2in_data['avg_speed']/road2in_data['mffs']

C:\Users\Tananett\AppData\Local\Temp/ipykernel_17128/1312273665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  road2in_data['portion_speed'] = road2in_data['avg_speed']/road2in_data['mffs']


In [140]:
x_predict = road2in_data[num_cols]

In [141]:
y_predict = gnb_model.predict(x_predict)

In [142]:
road2in_data['cluster'] = y_predict

C:\Users\Tananett\AppData\Local\Temp/ipykernel_17128/212174199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  road2in_data['cluster'] = y_predict


In [143]:
y_predict_proba = gnb_model.predict_proba(x_predict)

In [144]:
problist = []
for i in y_predict_proba :
    problist.append(round(i[sus_cluster],3))


In [145]:
road2in_data['prob'] = problist

C:\Users\Tananett\AppData\Local\Temp/ipykernel_17128/3409130349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  road2in_data['prob'] = problist


In [146]:
road2in_data = road2in_data[(road2in_data['cluster']==sus_cluster) & (road2in_data['prob']>=0.8) & (road2in_data['all_units']>=1)]

In [147]:
road2in_data

,datetime,road_number,km,direction,all_units,inflow_units,outflow_unit,samecell_units,avg_speed,max_speed,avg_traveltime,max_traveltime,mffs,lat,lon,portion_speed,cluster,prob
1268,2022-04-28 20:40:30,2,23,in,2,0,1,2,0,0,0,0,112,14.625379,101.099732,0.000000,1,1.000
1333,2022-04-28 20:40:30,2,57,in,1,0,3,1,0,0,0,0,132,14.660701,101.394109,0.000000,1,1.000
1387,2022-04-28 20:40:30,2,84,in,1,0,0,1,16,16,0,0,122,14.827820,101.546753,0.131148,1,0.986
1505,2022-04-28 20:40:30,2,144,in,1,0,1,1,1,1,0,0,87,14.963840,102.064058,0.011494,1,1.000
1509,2022-04-28 20:40:30,2,148,in,1,1,2,0,3,3,0,0,101,14.979692,102.088000,0.029703,1,1.000
1512,2022-04-28 20:40:30,2,155,in,1,0,0,1,0,0,0,0,135,15.021292,102.133879,0.000000,1,1.000
1574,2022-04-28 20:40:30,2,191,in,1,0,1,1,0,0,0,0,121,15.226275,102.367006,0.000000,1,1.000
1580,2022-04-28 20:40:30,2,194,in,1,1,3,0,0,0,79,79,145,15.244192,102.387928,0.000000,1,1.000
1694,2022-04-28 20:40:30,2,253,in,1,0,5,1,0,0,0,0,119,15.724166,102.592935,0.000000,1,1.000
1974,2022-04-28 20:40:30,2,411,in,1,1,2,0,0,0,0,0,104,17.058413,102.920818,0.000000,1,1.000


In [148]:
post_data = road2in_data[['road_number','km','direction','inflow_units','outflow_unit','samecell_units','all_units','avg_speed','lat','lon','datetime']]

In [149]:
post_data = post_data.values.tolist()

In [150]:
post_data

[[2, 23, 'in', 0, 1, 2, 2, 0, 14.6253785, 101.0997321, '2022-04-28 20:40:30'],
 [2, 57, 'in', 0, 3, 1, 1, 0, 14.66070063, 101.3941094, '2022-04-28 20:40:30'],
 [2,
  84,
  'in',
  0,
  0,
  1,
  1,
  16,
  14.82782034,
  101.5467535,
  '2022-04-28 20:40:30'],
 [2,
  144,
  'in',
  0,
  1,
  1,
  1,
  1,
  14.96384003,
  102.0640578,
  '2022-04-28 20:40:30'],
 [2,
  148,
  'in',
  1,
  2,
  0,
  1,
  3,
  14.97969219,
  102.0879996,
  '2022-04-28 20:40:30'],
 [2,
  155,
  'in',
  0,
  0,
  1,
  1,
  0,
  15.02129224,
  102.1338786,
  '2022-04-28 20:40:30'],
 [2,
  191,
  'in',
  0,
  1,
  1,
  1,
  0,
  15.22627509,
  102.3670062,
  '2022-04-28 20:40:30'],
 [2,
  194,
  'in',
  1,
  3,
  0,
  1,
  0,
  15.24419184,
  102.3879281,
  '2022-04-28 20:40:30'],
 [2,
  253,
  'in',
  0,
  5,
  1,
  1,
  0,
  15.72416561,
  102.5929353,
  '2022-04-28 20:40:30'],
 [2,
  411,
  'in',
  1,
  2,
  0,
  1,
  0,
  17.05841279,
  102.9208178,
  '2022-04-28 20:40:30'],
 [2,
  464,
  'in',
  0,
  1,
  1

In [151]:
# url = 'https://tad-api-v1.herokuapp.com/api/accident/bulk'
# # headers = {'Content-type': 'application/json','Accept': 'application/json'}
# result = requests.post(url, json=post_data)

In [152]:
# print(result.status_code)
# print(result.reason)